# Criação de arquivos para submissão

## import das bibliotecas e data set

Preparação do conjunto de testes oficial da competição (`df_test_processed`). 

* **Isolamento de Identificadores (`test_ids`):** Separamos a coluna de IDs para que não seja interpretada como variável explicativa pelo modelo, mas mantemos o mapeamento exato para a geração do arquivo de submissão exigido pelo Kaggle.
* **Alinhamento Vetorial (`X_test_final`):** Garantimos que a matriz preditiva do teste possua estritamente a mesma ordem e cardinalidade de colunas do conjunto de treinamento (`X_train`). Esse alinhamento estrutural é um pré-requisito crítico para evitar que os algoritmos apliquem pesos matemáticos a variáveis incorretas durante a fase de inferência cega.

In [1]:
import pandas as pd
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import shap


from sklearn.metrics import (
    classification_report, 
    confusion_matrix
)

# 1. Carregamento e Preparação de Pastas
path_dados = '../data/'
path_models = '../models/'
submission_dir = '../submission/'
submission_dir_others = os.path.join(submission_dir, 'others')
if not os.path.exists(submission_dir_others):
    os.makedirs(submission_dir_others)
folder_path = os.path.join(path_models, 'new_models')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

X_train = pd.read_csv(os.path.join(path_dados, 'X_train.csv'))
X_val = pd.read_csv(os.path.join(path_dados, 'X_val.csv'))
y_train = pd.read_csv(os.path.join(path_dados, 'y_train.csv')).values.ravel()
y_val = pd.read_csv(os.path.join(path_dados, 'y_val.csv')).values.ravel()
X_test_final = pd.read_csv(os.path.join(path_dados, 'X_test_final.csv'))

test_ids = pd.read_csv(os.path.join(path_dados, 'test.csv'))['id']

# Inicialização do dicionário global de resultados (executar apenas uma vez)
if 'all_results' not in locals():
    all_results = {}

print(f"Arquivos carregados. {X_train.shape[0]} amostras de treino, {X_val.shape[0]} amostras de validação e {X_test_final.shape[0]} amostras de teste.")

c:\Users\Guilherme\anaconda3\envs\LIGIA_indv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Arquivos carregados. 182276 amostras de treino, 45569 amostras de validação e 56962 amostras de teste.


## 1 - Rodar o melhor modelo com os dados de teste do kaggle para fazer a submissão e realizar outros testes

A seleção para a construção do arquivo oficial de submissão recaiu sobre a **Rede Neural (MLP Classifier)**, que se consolidou como o modelo campeão em nossa bateria de validação local unificada, demonstrando a melhor curva ROC-AUC. 

Para atestar a robustez dessa escolha e garantir que não estávamos diante de um cenário de *overfitting* no conjunto de validação, adotamos uma abordagem de validação empírica cruzada: submetemos experimentalmente na plataforma do Kaggle os resultados dos **3 modelos de melhor performance** do nosso ambiente local (ex: LightGBM, CatBoost e MLP). 

**Resultado Oficial:**
Consolidando nossa hipótese analítica, a Rede Neural provou ter a maior capacidade de generalização em dados estritamente não vistos (Conjunto de Teste Privado/Público da competição). A topologia MLP superou os métodos baseados em árvores (*Ensembles*) e manteve o melhor resultado prático, atingindo um **Kaggle Public Score de 0.99208 (ROC-AUC)**.

### 1.1 - Rodando o melhor modelo

In [ ]:
best_model_name = "Neural net.pkl"
best_model_obj = joblib.load(f"../models/best_model_fraud.pkl")
print(f"\n>>> 🚀 Iniciando etapa final para o modelo campeão: {best_model_name}...")

# 1. GERAÇÃO DO ARQUIVO DE SUBMISSÃO

print(f">>> 1. Gerando submission.csv...")

cols_used = X_train.columns.tolist()

if isinstance(X_test_final, pd.DataFrame):
    X_test_submission = X_test_final[cols_used]
else:
    X_test_submission = pd.DataFrame(X_test_final, columns=cols_used)

submission_probs = best_model_obj.predict_proba(X_test_submission)[:, 1]

submission = pd.DataFrame({
    'id': test_ids,
    'target': submission_probs
})

# Salvamento
submission_file = os.path.join(submission_dir, 'submission.csv')
submission.to_csv(submission_file, index=False)
print(f"✅ Arquivo '{submission_file}' salvo com sucesso!")
print(f"   - Total de linhas: {len(submission)}")
print(f"   - Colunas: {submission.columns.tolist()}")
print(submission.head())


>>> 🚀 Iniciando etapa final para o modelo campeão: Neural net.pkl...
>>> 1. Gerando submission.csv...
✅ Arquivo '../submission/submission.csv' salvo com sucesso!
   - Total de linhas: 56962
   - Colunas: ['id', 'target']
       id    target
0  263021  0.000065
1   11379  0.000050
2  147284  0.000090
3  219440  0.000122
4   36940  0.003133


### 1.2 - Testes de outros modelos promissores

Apesar de a Neural Net ter consolidado a liderança nas métricas oficiais, realizamos a extração e a geração das predições cegas para os **três melhores modelos secundários** que treinamos. 

* **Foco nos Melhores e Ensembles:** Filtrar as submissões apenas para este seleto grupo (ao invés de todos os modelos testados) otimiza nosso tempo e nos permite comparar as variações mais competitivas no *Leaderboard* do Kaggle. Além disso, viabiliza a futura criação de estratégias de votação conjunta (*blending*) usando apenas algoritmos de alta performance.
* **O Destaque entre as Alternativas:** Dentre esse Top 3 secundário, **o melhor e que esteve mais próximo da Neural Net foi o XGBoost**. Ele conseguiu equilibrar a punição de erros na classe minoritária e entregar uma generalização bastante robusta na matriz de testes, isolando-se como a nossa principal alternativa baseada em árvores de decisão.

#### 1.2.1- lightGBM

In [ ]:
best_model_name = 'LightGBM' 
best_model_obj = joblib.load(f"../models/new_models/modelo_lightgbm.pkl")
print(f"\n>>> 🚀 Iniciando etapa final para o modelo campeão: {best_model_name}...")

# 1. GERAÇÃO DO ARQUIVO DE SUBMISSÃO

print(f">>> 1. Gerando submission.csv...")

cols_used = X_train.columns.tolist()

if isinstance(X_test_final, pd.DataFrame):
    X_test_submission = X_test_final[cols_used]
else:
    X_test_submission = pd.DataFrame(X_test_final, columns=cols_used)

submission_probs = best_model_obj.predict_proba(X_test_submission)[:, 1]

submission = pd.DataFrame({
    'id': test_ids,
    'target': submission_probs
})

submission_file = os.path.join(submission_dir_others, 'submission_Light.csv')
submission.to_csv(submission_file, index=False)
print(f"✅ Arquivo '{submission_file}' salvo com sucesso!")
print(f"   - Total de linhas: {len(submission)}")
print(f"   - Colunas: {submission.columns.tolist()}")
print(submission.head()) 


>>> 🚀 Iniciando etapa final para o modelo campeão: LightGBM...
>>> 1. Gerando submission.csv...
✅ Arquivo '../submission/others\submission_Light.csv' salvo com sucesso!
   - Total de linhas: 56962
   - Colunas: ['id', 'target']
       id    target
0  263021  0.000011
1   11379  0.000007
2  147284  0.000036
3  219440  0.000004
4   36940  0.000604


#### 1.2.2 - XGBoost

In [ ]:
best_model_name = 'XGBoost'
best_model_obj = joblib.load(f"../models/new_models/modelo_xgboost.pkl")
print(f"\n>>> 🚀 Iniciando etapa final para o modelo campeão: {best_model_name}...")

# 1. GERAÇÃO DO ARQUIVO DE SUBMISSÃO 
print(f">>> 1. Gerando submission.csv...")

cols_used = X_train.columns.tolist()

if isinstance(X_test_final, pd.DataFrame):
    X_test_submission = X_test_final[cols_used]
else:
    X_test_submission = pd.DataFrame(X_test_final, columns=cols_used)

submission_probs = best_model_obj.predict_proba(X_test_submission)[:, 1]

submission = pd.DataFrame({
    'id': test_ids,
    'target': submission_probs
})

submission_file = os.path.join(submission_dir_others, 'submission_XGB.csv')
submission.to_csv(submission_file, index=False)
print(f"✅ Arquivo '{submission_file}' salvo com sucesso!")
print(f"   - Total de linhas: {len(submission)}")
print(f"   - Colunas: {submission.columns.tolist()}")
print(submission.head())


>>> 🚀 Iniciando etapa final para o modelo campeão: XGBoost...
>>> 1. Gerando submission.csv...
✅ Arquivo '../submission/others\submission_XGB.csv' salvo com sucesso!
   - Total de linhas: 56962
   - Colunas: ['id', 'target']
       id    target
0  263021  0.015230
1   11379  0.015072
2  147284  0.018764
3  219440  0.013622
4   36940  0.047000


#### 1.2.3 - Catboost

In [ ]:
best_model_name = 'CatBoost' 
best_model_obj = joblib.load(f"../models/new_models/modelo_catboost.pkl")
print(f"\n>>> 🚀 Iniciando etapa final para o modelo campeão: {best_model_name}...")

# 1. GERAÇÃO DO ARQUIVO DE SUBMISSÃO 
print(f">>> 1. Gerando submission.csv...")

cols_used = X_train.columns.tolist()

if isinstance(X_test_final, pd.DataFrame):
    X_test_submission = X_test_final[cols_used]
else:
    X_test_submission = pd.DataFrame(X_test_final, columns=cols_used)

submission_probs = best_model_obj.predict_proba(X_test_submission)[:, 1]

submission = pd.DataFrame({
    'id': test_ids,
    'target': submission_probs
})

submission_file = os.path.join(submission_dir_others, 'submission_Cat.csv')
submission.to_csv(submission_file, index=False)
print(f"✅ Arquivo '{submission_file}' salvo com sucesso!")
print(f"   - Total de linhas: {len(submission)}")
print(f"   - Colunas: {submission.columns.tolist()}")
print(submission.head()) 


>>> 🚀 Iniciando etapa final para o modelo campeão: CatBoost...
>>> 1. Gerando submission.csv...
✅ Arquivo '../submission/others\submission_Cat.csv' salvo com sucesso!
   - Total de linhas: 56962
   - Colunas: ['id', 'target']
       id    target
0  263021  0.000159
1   11379  0.000350
2  147284  0.037921
3  219440  0.000019
4   36940  0.003354
